In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [ ]:
textwidth=360/72

In [ ]:
def model(n, p, A):
    return A * n**3 / p

In [ ]:
def parse_mmul_out(fp):
    results = []
    with open(fp) as f:
        for line in f:
            m = re.match('N=(\d+) TH=(\d+)', line)
            if m:
                N = m.group(1)
                th = m.group(2)

            m = re.search(
                r'(OpenMP|ThreadPool|LFThreadPool|threadpool11): (\d+\.\d+e[+-]\d+) +(\d+\.\d+e[+-]\d+) +(\d+\.\d+e[+-]\d+)', line)
            if m:
                results.append({'N': int(N),
                                'num_threads': int(th),
                                'library': str(m.group(1)),
                                'minTime': float(m.group(2)),
                                'meanTime': float(m.group(3)),
                                'maxTime': float(m.group(4))})
    return pd.DataFrame(results)

In [ ]:
macres = parse_mmul_out('./mmul_macmini.txt')
bwres = parse_mmul_out('./mmul_bw2.txt')

In [ ]:
sns.set_style('ticks')
sns.set_context('paper')
sns.set_color_codes()

In [ ]:
bwres.head()

In [ ]:
fg = sns.FacetGrid(macres, hue='library', col='N', size=5, sharey=False, col_wrap=2)
fg.map(plt.plot, 'num_threads', 'minTime').add_legend()
# plt.semilogy()

In [ ]:
def myplot(*args, **kwargs):
    plt.plot(*args, **kwargs)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 2))
#     plt.loglog(basex=2, basey=10)

plt.figure(figsize=(6,6))
fg = sns.FacetGrid(bwres, hue='library', col='N', size=textwidth/3, sharey=False, col_wrap=3,
                   hue_kws={'marker': ['o', 's', 'v', '^'], 'linestyle': ['-', '--', '-.', ':']})
fg.map(myplot, 'num_threads', 'minTime')
# fg.add_legend(title='Library')
fg.set_axis_labels('Number of threads', 'Min time [s]')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 0), bbox_transform=plt.gcf().transFigure,
           ncol=2, frameon=False)

# plt.savefig('mmul_times.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(3, 3*0.8))
d = bwres[(bwres.N == 2000) & (bwres.library == 'OpenMP')]
plt.plot(d.num_threads, d.minTime, 'o', label='OpenMP')
xs = np.logspace(1, 5, 100, base=2)
plt.plot(xs, model(2000, xs, A=2.7e-8), 'b-', label='Model')
plt.legend()
sns.despine()
plt.loglog(basex=2)
plt.xlim(2**0.9, 2**5.1)
plt.xlabel('Number of threads')
plt.ylabel('Min time [s]')
plt.tight_layout()
# plt.savefig('modelfit.pdf')

In [ ]:
from itertools import cycle
mkr = cycle(['o-', 's--', 'v-.', '^:'])

N = 2000

plt.figure(figsize=(3, 3*0.8))
d = bwres[(bwres.N == N)]
for i, df in d.groupby('library'):
    plt.plot(df.num_threads, df.minTime / model(N, df.num_threads, A=2.7e-8), next(mkr), label=i)
    
    plt.legend(loc=2, frameon=True)
    plt.xlim(2**(0.9), 2**5.1)
#     plt.hlines(1.0, 2, 2**5, linewidth=1)

    plt.semilogx(basex=2)
    plt.grid(True, which='both')
#     sns.despine()
    
    plt.ylabel('Ratio to model')
    plt.xlabel('Number of threads')
    
    plt.tight_layout()
    
    plt.savefig('mmul_ratios.pdf')